In [1]:
from finlib.all import *

In [2]:
df = pd.read_csv('TXF20210101_20210731.csv')
df.columns = df.columns.str.lower()
df.index = pd.to_datetime(df.date + ' ' + df.time)
df.index.name = 'datetime'
df = df.between_time('08:45', '13:45')
df

,date,time,open,high,low,close,volume
datetime,,,,,,,
2021-01-04 08:46:00,2021/01/04,08:46:00,14640,14647,14635,14646,1530
2021-01-04 08:47:00,2021/01/04,08:47:00,14647,14655,14642,14650,691
2021-01-04 08:48:00,2021/01/04,08:48:00,14651,14652,14643,14646,322
2021-01-04 08:49:00,2021/01/04,08:49:00,14647,14656,14646,14653,309
2021-01-04 08:50:00,2021/01/04,08:50:00,14653,14654,14650,14652,199
...,...,...,...,...,...,...,...
2021-07-30 13:41:00,2021/07/30,13:41:00,17159,17163,17159,17162,350
2021-07-30 13:42:00,2021/07/30,13:42:00,17164,17170,17163,17168,412
2021-07-30 13:43:00,2021/07/30,13:43:00,17167,17173,17167,17173,466


In [3]:
df = (
    df
    .pipe(cal_sma, 'close', (5, 10, 20, 60, 120))
    .pipe(cal_sma, 'volume', (5, 10), suffix='_v')
    .pipe(cal_bbands)
    .pipe(cal_vwap, '5H')
    .pipe(cal_macd)
    .pipe(cal_kdj)
    .pipe(cal_cross, 'k9', 'd9', 'd9')
)

In [4]:
_df = df.iloc[-300:]

In [5]:
kline = draw_klines(_df)
kline.render_notebook()

In [54]:
f = plot_random_date4(_df.iloc[:20], direct_render=False, plate3_line_keys=['k9', 'd9'],
                      plate1_line_key=['sma5', 'sma60'],
channel_key='bbands',
vwap=True)

In [55]:
f.options['series']

[{'type': 'candlestick',
  'name': 'Kline',
  'xAxisIndex': 0,
  'yAxisIndex': 0,
  'data': [[15420, 15393, 15388, 15422],
   [15391, 15391, 15390, 15398],
   [15391, 15402, 15383, 15405],
   [15403, 15406, 15397, 15412],
   [15405, 15414, 15402, 15421],
   [15413, 15422, 15413, 15424],
   [15423, 15418, 15416, 15428],
   [15418, 15421, 15417, 15427],
   [15421, 15416, 15412, 15421],
   [15415, 15408, 15403, 15415],
   [15406, 15411, 15405, 15414],
   [15411, 15410, 15404, 15412],
   [15410, 15420, 15405, 15421],
   [15420, 15417, 15414, 15420],
   [15416, 15416, 15414, 15421],
   [15417, 15416, 15404, 15425],
   [15417, 15434, 15416, 15434],
   [15432, 15444, 15430, 15448],
   [15444, 15437, 15432, 15448],
   [15437, 15428, 15417, 15439]],
  'markPoint': <pyecharts.options.series_options.MarkPointOpts at 0x23d612f9d48>,
  'markLine': None,
  'tooltip': None,
  'itemStyle': <pyecharts.options.series_options.ItemStyleOpts at 0x23d612f9f08>},
 {'type': 'line',
  'name': 'sma5',
  'connec

In [26]:
f.options

{'backgroundColor': '#23262F',
 'animation': False,
 'animationThreshold': 2000,
 'animationDuration': 1000,
 'animationEasing': 'cubicOut',
 'animationDelay': 0,
 'animationDurationUpdate': 300,
 'animationEasingUpdate': 'cubicOut',
 'animationDelayUpdate': 0,
 'series': [{'type': 'candlestick',
   'name': 'Kline',
   'xAxisIndex': 0,
   'yAxisIndex': 0,
   'data': [[15420, 15393, 15388, 15422],
    [15391, 15391, 15390, 15398],
    [15391, 15402, 15383, 15405],
    [15403, 15406, 15397, 15412],
    [15405, 15414, 15402, 15421],
    [15413, 15422, 15413, 15424],
    [15423, 15418, 15416, 15428],
    [15418, 15421, 15417, 15427],
    [15421, 15416, 15412, 15421],
    [15415, 15408, 15403, 15415],
    [15406, 15411, 15405, 15414],
    [15411, 15410, 15404, 15412],
    [15410, 15420, 15405, 15421],
    [15420, 15417, 15414, 15420],
    [15416, 15416, 15414, 15421],
    [15417, 15416, 15404, 15425],
    [15417, 15434, 15416, 15434],
    [15432, 15444, 15430, 15448],
    [15444, 15437, 154

In [19]:
from random import randrange

from flask.json import jsonify
from flask import Flask, render_template

from pyecharts import options as opts
from pyecharts.charts import Line


app = Flask(__name__, static_folder="templates")

_df = df[1500:1800]
idx = df.index.get_loc(_df.index[-1])

@app.route("/")
def index():
    return render_template("single_kline_index.html")


@app.route("/barChart")
def get_line_chart():
    global _df, idx
    kline = draw_klines(_df)
    return kline.dump_options_with_quotes()

@app.route("/lineDynamicData")
def update_line_data():
    global idx
    #idx = idx + pd.Timedelta('1min')
    idx += 1
    index = df.index[idx]
    ydata = df.loc[index, ['open', 'close', 'low', 'high']].astype(np.float32).tolist()
    
    
    return jsonify({"xdata": index, "ydata": ydata}) # 'open', 'close', 'low', 'high'


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Feb/2022 11:15:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:15:57] "GET /barChart HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:15:57] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:15:58] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:15:58] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:15:59] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:15:59] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:16:00] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:16:00] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:16:01] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:16:01] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:16:02] "GET /lineDynamicData HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2022 11:16:02] "GET /lineDynamicData HTT

In [368]:
kline.get_options()['xAxis'][0]['data']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [370]:
kline.get_options()['series'][1]

{'type': 'line',
 'name': 'sma5',
 'connectNulls': False,
 'symbolSize': 0.1,
 'showSymbol': True,
 'smooth': True,
 'clip': True,
 'step': False,
 'data': [[0, 17350.4],
  [1, 17329.2],
  [2, 17303.8],
  [3, 17279.4],
  [4, 17255.0],
  [5, 17247.2],
  [6, 17247.6],
  [7, 17250.6],
  [8, 17251.8],
  [9, 17252.8],
  [10, 17251.8],
  [11, 17248.6],
  [12, 17246.4],
  [13, 17245.8],
  [14, 17245.6],
  [15, 17249.0],
  [16, 17252.2],
  [17, 17257.0],
  [18, 17262.4],
  [19, 17269.0],
  [20, 17273.8],
  [21, 17280.2],
  [22, 17282.8],
  [23, 17285.0],
  [24, 17282.4],
  [25, 17278.4],
  [26, 17273.6],
  [27, 17272.4],
  [28, 17269.8],
  [29, 17270.4],
  [30, 17273.2],
  [31, 17276.2],
  [32, 17278.2],
  [33, 17280.2],
  [34, 17278.4],
  [35, 17274.6],
  [36, 17267.4],
  [37, 17258.6],
  [38, 17254.2],
  [39, 17251.8],
  [40, 17252.2],
  [41, 17255.8],
  [42, 17259.2],
  [43, 17255.6],
  [44, 17251.6],
  [45, 17248.4],
  [46, 17245.0],
  [47, 17244.2],
  [48, 17246.6],
  [49, 17250.6],
  [50

In [360]:
kline.get_options()['series'][0]

{'type': 'candlestick',
 'name': 'Kline',
 'data': [[17285, 17289, 17263, 17295],
  [17288, 17257, 17256, 17288],
  [17256, 17239, 17223, 17257],
  [17238, 17243, 17233, 17245],
  [17243, 17247, 17239, 17249],
  [17248, 17250, 17242, 17250],
  [17249, 17259, 17246, 17259],
  [17256, 17254, 17254, 17259],
  [17253, 17249, 17246, 17253],
  [17249, 17252, 17248, 17255],
  [17251, 17245, 17243, 17251],
  [17243, 17243, 17241, 17247],
  [17242, 17243, 17240, 17243],
  [17242, 17246, 17241, 17246],
  [17246, 17251, 17244, 17254],
  [17252, 17262, 17244, 17264],
  [17260, 17259, 17251, 17264],
  [17258, 17267, 17245, 17268],
  [17268, 17273, 17268, 17275],
  [17273, 17284, 17272, 17287],
  [17285, 17286, 17282, 17289],
  [17285, 17291, 17285, 17293],
  [17292, 17280, 17275, 17292],
  [17280, 17284, 17275, 17287],
  [17285, 17271, 17270, 17287],
  [17271, 17266, 17258, 17272],
  [17265, 17267, 17263, 17271],
  [17267, 17274, 17261, 17277],
  [17275, 17271, 17271, 17280],
  [17271, 17274, 17266

In [364]:
c.get_options()['series']

[{'type': 'line',
  'connectNulls': False,
  'symbolSize': 4,
  'showSymbol': True,
  'smooth': True,
  'clip': True,
  'step': False,
  'data': [['0', 53],
   ['1', 68],
   ['2', 56],
   ['3', 76],
   ['4', 67],
   ['5', 63],
   ['6', 76],
   ['7', 77],
   ['8', 58],
   ['9', 79]],
  'hoverAnimation': True,
  'label': <pyecharts.options.series_options.LabelOpts at 0x1b2290b4408>,
  'lineStyle': <pyecharts.options.series_options.LineStyleOpts at 0x1b2100a12c8>,
  'areaStyle': <pyecharts.options.series_options.AreaStyleOpts at 0x1b2100a1308>,
  'zlevel': 0,
  'z': 0}]

In [272]:
c.get_options()['xAxis']

[{'type': 'value',
  'show': True,
  'scale': False,
  'nameLocation': 'end',
  'nameGap': 15,
  'gridIndex': 0,
  'inverse': False,
  'offset': 0,
  'splitNumber': 5,
  'minInterval': 0,
  'splitLine': <pyecharts.options.series_options.SplitLineOpts at 0x1b210055fc8>,
  'data': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']}]

In [62]:
kline.get_options()['series'][0]

{'type': 'candlestick',
 'name': 'Kline',
 'data': [[17285, 17289, 17263, 17295],
  [17288, 17257, 17256, 17288],
  [17256, 17239, 17223, 17257],
  [17238, 17243, 17233, 17245],
  [17243, 17247, 17239, 17249],
  [17248, 17250, 17242, 17250],
  [17249, 17259, 17246, 17259],
  [17256, 17254, 17254, 17259],
  [17253, 17249, 17246, 17253],
  [17249, 17252, 17248, 17255],
  [17251, 17245, 17243, 17251],
  [17243, 17243, 17241, 17247],
  [17242, 17243, 17240, 17243],
  [17242, 17246, 17241, 17246],
  [17246, 17251, 17244, 17254],
  [17252, 17262, 17244, 17264],
  [17260, 17259, 17251, 17264],
  [17258, 17267, 17245, 17268],
  [17268, 17273, 17268, 17275],
  [17273, 17284, 17272, 17287],
  [17285, 17286, 17282, 17289],
  [17285, 17291, 17285, 17293],
  [17292, 17280, 17275, 17292],
  [17280, 17284, 17275, 17287],
  [17285, 17271, 17270, 17287],
  [17271, 17266, 17258, 17272],
  [17265, 17267, 17263, 17271],
  [17267, 17274, 17261, 17277],
  [17275, 17271, 17271, 17280],
  [17271, 17274, 17266